# Sandbox
Notebook to test code before real implementation

In [4]:
################ ONLY PYTHON 2 #########################
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations, chain
import argparse
import time
import os
import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics
import pandas as pd
import psutil
import pickle
from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import os
import psutil
import pickle
from pybdm import BDM
from pybdm.utils import decompose_dataset
from pybdm.partitions import PartitionIgnore
from joblib import Parallel, delayed
from joblib import parallel_backend
from data.algorithms import PerturbationExperiment, NodePerturbationExperiment
import math
import datetime as dt
from itertools import product

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/juan/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Revision of data structures

In [3]:
filename = 'data/data_structures/DS/DS_real_DSE_9702_PF_5_NPF_3_genes_16271_drugs_639_se_964'

In [4]:
with open(filename, 'rb') as f:
    DS = pickle.load(f)
    for key in DS.keys():
        globals()[key]=DS[key]
        print(key,"Imported successfully")

gene2idx Imported successfully
drug2idx Imported successfully
se_mono_name2idx Imported successfully
se_combo_name2idx Imported successfully
ddi_adj_list Imported successfully
ddi_degrees_list Imported successfully
dti_adj Imported successfully
ppi_adj Imported successfully
ppi_degrees Imported successfully
drug_feat Imported successfully
prot_feat Imported successfully
norm_prot_feat Imported successfully


In [17]:
ses = ['Mumps','carbuncle','coccydynia','Bleeding','disorder Renal','emesis']
for i, j in se_combo_name2idx.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if i in ses:
        print(i,j)

emesis 9
Bleeding 58
disorder Renal 150
Mumps 920
carbuncle 956
coccydynia 957


In [14]:
print(se_combo_name2idx)

{'arterial pressure NOS decreased': 0, 'anaemia': 1, 'Difficulty breathing': 2, 'nausea': 3, 'neumonia': 4, 'Fatigue': 5, 'diarrhea': 6, 'Pain': 7, 'asthenia': 8, 'emesis': 9, 'edema extremities': 10, 'body temperature increased': 11, 'pleural pain': 12, 'abdominal pain': 13, 'Hypoventilation': 14, 'chest pain': 15, 'dizziness': 16, 'Back Ache': 17, 'Head ache': 18, 'confusion': 19, 'High blood pressure': 20, 'dehydration': 21, 'kidney failure': 22, 'Anxiety': 23, 'hyperglycaemia': 24, 'loss of weight': 25, 'edema': 26, 'Anorexia': 27, 'Aching joints': 28, 'acute kidney failure': 29, 'neuropathy': 30, 'Extremity pain': 31, 'Cough': 32, 'Infection Urinary Tract': 33, 'constipated': 34, 'heart rate increased': 35, 'weight gain': 36, 'thrombocytopenia': 37, 'Pleural Effusion': 38, 'sepsis': 39, 'Cardiac decompensation': 40, 'heart attack': 41, 'respiratory failure': 42, 'eruption': 43, 'AFIB': 44, 'drowsiness': 45, 'insomnia': 46, 'muscle spasm': 47, 'Neutropenia': 48, 'apoplexy': 49, 'bl

In [5]:
print(np.shape(adj_mats_orig[0,1][0].transpose()))
print(type(adj_mats_orig[0,1][0].transpose()))

(630, 16269)
<class 'scipy.sparse.csc.csc_matrix'>


In [6]:
# Checking if ppi and ppi transpose are equal
(adj_mats_orig[0,0][1]!=adj_mats_orig[0,0][0]).nnz==0

True

In [7]:
# Checking if dti and dti transpose are equal
(adj_mats_orig[0,1][0]!=adj_mats_orig[1,0][0].transpose()).nnz==0

True

In [8]:
# Checking if ddi and ddi transpose are equal
print((adj_mats_orig[1,1][0]!=adj_mats_orig[1,1][6]).nnz==0)
print((adj_mats_orig[1,1][1]!=adj_mats_orig[1,1][7]).nnz==0)
print((adj_mats_orig[1,1][2]!=adj_mats_orig[1,1][8]).nnz==0)
print((adj_mats_orig[1,1][3]!=adj_mats_orig[1,1][9]).nnz==0)
print((adj_mats_orig[1,1][4]!=adj_mats_orig[1,1][10]).nnz==0)
print((adj_mats_orig[1,1][5]!=adj_mats_orig[1,1][11]).nnz==0)

True
True
True
True
True
True


In [18]:
print(np.shape(feat[1][1]))

(186104,)


In [17]:
feat

{0: (array([[    0,     0],
         [    0,     1],
         [    0,     3],
         ...,
         [16268,     3],
         [16268,     4],
         [16268,     5]], dtype=int32),
  array([-1.09692919e+03,  3.64008518e-01,  1.46930882e+03, ...,
         -2.93100170e+01,  7.70666054e+00, -7.40744144e+00]),
  (16269, 6)),
 1: (array([[   0,    6],
         [   0,    9],
         [   0,   10],
         ...,
         [ 629, 9701],
         [ 629, 9702],
         [ 629, 9708]], dtype=int32),
  array([ 1.        ,  1.        ,  1.        , ...,  0.07420517,
          0.07704289, -0.03897962]),
  (630, 9709))}

In [19]:
a = []
a.append((3)(7))

TypeError: 'int' object is not callable